In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Get the URL of the website with election results 
- <i>Here we are looking at <b>UNOFFICIAL</b> results collected by this site </i>

In [2]:
url = "https://www.tibetsun.com/elections/sikyong-2016-final-round-results#election-results"
req = requests.get(url)

data = req.text

soup = BeautifulSoup(data)

### Read how BeautifulSoup (parser) works [here](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree)

- Here I'm getting the data collection ready by creating these List variables
- First data row happens to be the overall total (votes)
- In few seconds, you will see below, the rest of the vote count is provided by location

In [3]:
#Overall Results 

location_total_vote_list = []
ls_vote_count_list = []
pt_vote_count_list = []

election_results = soup.find("table", class_="election-results elections-page")

location_total_vote_list.append("Total Overall")


### Read the Beautiful Soup documentation to find out how to target specific HTML tags
### Here I'm find all the "TR" Table row from within the Table data in HTML from the site

### In Particular, I'm finding for all "TR", the "nm" or Name of candidates.
### And alos, total "vt" or votes the candidate got

for row in election_results.find_all('tr'):
    if("Lobsang" in row.find(class_="nm").string):
        ls_vote_count_list.append(row.find(class_="vt").string)
    elif ("Penpa" in row.find(class_="nm").string):
        pt_vote_count_list.append(row.find(class_="vt").string)
        
### TESTING

    #print(row.find(class_ ="nm").string)
    #print(row.find(class_="vt").string)
        

In [4]:
def test_results():
    print(len(location_total_vote_list))
    print(len(ls_vote_count_list))
    print(len(pt_vote_count_list))
    

#### Here we will be collecting or scraping all the vote count for each location
- We are using beautiful soup to automate the data collection (bulk)

In [5]:
### HERE finding all the Table Rows with vote total for each "Location"
#Results by location - find all table (HTML) rows with "Location" names

results_by_location = soup.find("div", class_="vote-results")


- We have the bulk data, let's iterate over each row or chunk of the data
- Each chunk contains, the names of candidates and their vote total 
- Unhide the comments to see the what's happening behind the scene

In [6]:
#Iterate through each "Location" data row in Table 
#Extract the total vote count for each candidate

for row in results_by_location.find_all("table"):
    #print(row.contents[1].find('h3'))
    #print(row.contents[2].find("h3").contents[0].string)
    
    location_total_vote_list.append(row.contents[2].find("h3").contents[0].string)
    #print("   Lobsang:", row.contents[4].find(class_="vt").string)
    
    ls_vote_count_list.append(row.contents[4].find(class_ ="vt").string)
    #print("   Penpa:", row.contents[6].find(class_="vt").string)
    
    pt_vote_count_list.append(row.contents[6].find(class_="vt").string)

In [7]:
#test_results()

In [8]:
def test_location_vote_count():

    for index in range(0, len(location_total_vote_list)):
        print(location_total_vote_list[index])
        print(" LS: ", ls_vote_count_list[index])
        print(" PT: ", pt_vote_count_list[index])


### Convert the List files into beautiful DataFrame
- This allows us to do manipulation, cleaning and more calculations much faster and easier

In [9]:
#Convert Lists to DataFrame (Panda)
vote_total_table = pd.DataFrame(
    {'Location':location_total_vote_list,
     'Lobsang Sangay': ls_vote_count_list,
     'Penpa Tsering': pt_vote_count_list
    })

- More cleaning to do, convert the "vote counts" which are "objects" (strings) into "integers"
- Without this conversion, we can not do math calculations or we will get errors (Type errors)

In [10]:
#Convert "vote count" which is String Object into "int"
#This allows calculations to be done
vote_total_table['Lobsang Sangay'] = vote_total_table['Lobsang Sangay'].str.replace(',','').astype('int')
vote_total_table['Penpa Tsering'] = vote_total_table['Penpa Tsering'].str.replace(',','').astype('int')

In [11]:
#Test show the first row with Overall Vote Count Data
vote_total_table.head(1)

,Location,Lobsang Sangay,Penpa Tsering
0,Total Overall,33234,24752


#### We are doing more cleaning here
- We have votes from big European regions
- But we need to calculate the overall Europe vote total

In [12]:
#Tabulate the EU data total
#All Table rows containing "Europe" in Location tag

eu = vote_total_table.query('Location.str.contains("Europe") and not(Location.str.contains("Overall"))', engine='python')
print("Lobsang Sangay: ", eu['Lobsang Sangay'].sum())
print("Penpa Tsering: ", eu['Penpa Tsering'].sum())

Lobsang Sangay:  3348
Penpa Tsering:  2076


#### Little bit more cleaning to do
- We have votes local communitites mostly in Australia and some in Asia
- But we need to calculate the overall "Australasia" vote total

In [13]:
#Tabulate the Austral_Asia data total
#All Table rows containing "Austra" in Location tag
vote_total_table.loc[vote_total_table.Location == "Australasia (overall)", 'Location'] = "Australasia (Overall)"

asia = vote_total_table.query('Location.str.contains("Austra") and not(Location.str.contains("Overall"))', engine='python')
print("Lobsang Sangay: ", asia['Lobsang Sangay'].sum())
print("Penpa Tsering: ", asia['Penpa Tsering'].sum())

asia


Lobsang Sangay:  589
Penpa Tsering:  426


,Location,Lobsang Sangay,Penpa Tsering
1,"Australasia (Adelaide, Australia)",12,4
2,"Australasia (Auckland, New Zealand)",11,4
3,"Australasia (Brisbane, Australia)",22,16
4,"Australasia (Canberra, Australia)",35,20
5,Australasia (Japan),34,19
6,"Australasia (Melbourne, Australia)",122,93
7,"Australasia (Newcastle, Australia)",16,4
8,"Australasia (Sydney, Australia)",266,236
9,Australasia (Taiwan),63,30
10,"Australasia (Whangarei, New Zealand)",8,0


In [14]:
asia_total = vote_total_table[vote_total_table['Location'].str.contains("Austra")]
asia_total

,Location,Lobsang Sangay,Penpa Tsering
1,"Australasia (Adelaide, Australia)",12,4
2,"Australasia (Auckland, New Zealand)",11,4
3,"Australasia (Brisbane, Australia)",22,16
4,"Australasia (Canberra, Australia)",35,20
5,Australasia (Japan),34,19
6,"Australasia (Melbourne, Australia)",122,93
7,"Australasia (Newcastle, Australia)",16,4
8,"Australasia (Sydney, Australia)",266,236
9,Australasia (Taiwan),63,30
10,"Australasia (Whangarei, New Zealand)",8,0


- "North America (overall)" already exists: 
- Modify and update: (overall) to "(Overall)" to make it consistent


In [15]:

vote_total_table.loc[vote_total_table.Location == "North America (overall)", 'Location'] = "North America (Overall)"

- Add "Europe" Overall to the dataframe (Vote_total_table)

In [16]:
### Europe Overall, add it to Vote_Total_Table Count

eu_total = vote_total_table[vote_total_table['Location'].str.contains("Europe")]
eu_total["Lobsang Sangay"].sum()
eu_total["Penpa Tsering"].sum()

vote_total_table =vote_total_table.append(pd.Series(['Europe (Overall)', eu_total["Lobsang Sangay"].sum(), eu_total["Penpa Tsering"].sum()], 
                                  index=['Location', 'Lobsang Sangay', 'Penpa Tsering']), ignore_index=True)


#### Create a South Asia Total

In [17]:
### South Asia
## Tabulate the South Asia Vote Count Total by filtering out: Europe, AustralaAsia and North America.
#Add this Total to Vote_Count_Total_Table
south_asia_total = vote_total_table.query('~Location.str.contains("Europe") and' 
                                          +'~Location.str.contains("Australa") and ~Location.str.contains("America")'
                                          +'and ~Location.str.contains("Overall")', 
                       engine='python')
#print(south_asia_total)
vote_total_table = vote_total_table.append(pd.Series(['South Asia (Overall)', south_asia_total["Lobsang Sangay"].sum(),
                                                     south_asia_total["Penpa Tsering"].sum()],
                                                    index=['Location', 'Lobsang Sangay', 'Penpa Tsering']), ignore_index=True)

#### Create India and Nepal (Subtotals)

In [18]:
###Nepal & India Only
##Tabulate total vote count for Nepal and India (separately)

nepal_total = vote_total_table.query('Location.str.contains("Nepal")', engine='python')

vote_total_table = vote_total_table.append(pd.Series(['Nepal (Overall)', nepal_total['Lobsang Sangay'].sum(), 
                                  nepal_total['Penpa Tsering'].sum()],
                                  index=['Location', 'Lobsang Sangay', 'Penpa Tsering']), 
                                   ignore_index=True)

india_total = south_asia_total.query('~Location.str.contains("Nepal")', engine='python')

vote_total_table = vote_total_table.append(pd.Series(['India (Overall)', india_total['Lobsang Sangay'].sum(),
                                                india_total['Penpa Tsering'].sum()], 
                                                      index=['Location', 'Lobsang Sangay', 'Penpa Tsering']), 
                                                    ignore_index = True);

In [19]:
nepal_total

,Location,Lobsang Sangay,Penpa Tsering
43,Nepal (Dhorpatan),87,21
44,Nepal (Kathmandu),2048,1464
45,Nepal (Pokhara Jampa Ling),257,91
46,Nepal (Pokhara Tashi Ling),217,63
47,Nepal (Pokhara Tashi Palkhiel),218,99
48,Nepal (Shawara sum),438,125


#### Create a new column "Vote Population %"
- This column calculates what portion of the total voting population does this region or communit represents

In [20]:
### Calculate the Total Vote Count, in order to calculate the candidate's vote count (%)

grand_total = vote_total_table['Lobsang Sangay'][0] + vote_total_table['Penpa Tsering'][0]
vote_total_table['Voter Population %'] = round(100*((vote_total_table['Lobsang Sangay'] + vote_total_table['Penpa Tsering']) / grand_total), 2)

- Create an additional column called "Lobsang Sangay %"
    - This column is created by calculating the winning candidate's Vote received %

In [21]:
### Calculate the Winning Candidate's performance for each location or region

vote_total_table['Lobsang Sangay %'] = round(100*(vote_total_table['Lobsang Sangay']/(vote_total_table['Lobsang Sangay'] + 
                                       vote_total_table['Penpa Tsering'])), 1)

#### Sort the Table by "Voter Population %" by descending order
- So we will see which regions or communities represents the bulk of the voting population

In [22]:
vote_total_table = vote_total_table.sort_values(by=['Voter Population %'], ascending=False)
vote_total_table.head(20)

,Location,Lobsang Sangay,Penpa Tsering,Voter Population %,Lobsang Sangay %
0,Total Overall,33234,24752,100.00,57.3
71,South Asia (Overall),26194,16820,74.18,60.9
73,India (Overall),22929,14957,65.34,60.5
57,North America (Overall),5059,3482,14.73,59.2
24,Dharamshala,3116,2627,9.90,54.3
70,Europe (Overall),3348,2076,9.35,61.7
72,Nepal (Overall),3265,1863,8.84,63.7
41,Mundgod,2931,1841,8.23,61.4
19,Bylakuppe (Lugsam),2621,1501,7.11,63.6
44,Nepal (Kathmandu),2048,1464,6.06,58.3


- Here we are using the sorted table to indentify where the winning candidate performance was subpar

In [23]:
### Laggards Region
laggards = vote_total_table.query('~Location.str.contains("Overall")', engine='python').sort_values(by=['Lobsang Sangay %'], ascending=True)
print(laggards[laggards["Lobsang Sangay %"] <= 57.3]["Voter Population %"].sum(), "%")                                                                                                
#print(temp)

laggards[laggards["Lobsang Sangay %"] <= 57.3]

## 37.5 %


35.83 %


,Location,Lobsang Sangay,Penpa Tsering,Voter Population %,Lobsang Sangay %
51,"North America (United States, Boston)",187,185,0.64,50.3
38,Mainpat,214,210,0.73,50.5
12,Bangalore (South zone),452,435,1.53,51.0
29,Gangtok,650,604,2.16,51.8
64,Shimla,245,227,0.81,51.9
42,Nainital,45,41,0.15,52.3
53,"North America (United States, Minnesota)",439,399,1.45,52.4
52,"North America (United States, California)",353,317,1.16,52.7
13,Bhandara,182,162,0.59,52.9
8,"Australasia (Sydney, Australia)",266,236,0.87,53.0


- Here we see the regions or communities where the winning candidate performed the best

In [24]:
###Best performer location
strong_holds = vote_total_table.query('~Location.str.contains("Overall")', engine='python').sort_values(by=['Lobsang Sangay %'], ascending=False).head(30)
print((strong_holds[strong_holds["Lobsang Sangay %"] >= 57.3]["Voter Population %"].sum(), "%"))
strong_holds[strong_holds["Lobsang Sangay %"] >= 57.3]


(21.759999999999998, '%')


,Location,Lobsang Sangay,Penpa Tsering,Voter Population %,Lobsang Sangay %
10,"Australasia (Whangarei, New Zealand)",8,0,0.01,100.0
66,Tashi Jong,236,11,0.43,95.5
62,Sataun (KKTS),57,7,0.11,89.1
15,Bir (Dege and Nangchen),563,97,1.14,85.3
40,Miao,671,127,1.38,84.1
43,Nepal (Dhorpatan),87,21,0.19,80.6
7,"Australasia (Newcastle, Australia)",16,4,0.03,80.0
48,Nepal (Shawara sum),438,125,0.97,77.8
46,Nepal (Pokhara Tashi Ling),217,63,0.48,77.5
67,Tezu,228,70,0.51,76.5


#### Vote Count by regionals 
- See the top performing regions
- Notice the "overall total" at the second last row

In [25]:
vote_total_table.query('Location.str.contains("Overall")', engine='python').sort_values(by=['Lobsang Sangay %'], 
                                                                                         ascending=False).head(20)

,Location,Lobsang Sangay,Penpa Tsering,Voter Population %,Lobsang Sangay %
72,Nepal (Overall),3265,1863,8.84,63.7
70,Europe (Overall),3348,2076,9.35,61.7
71,South Asia (Overall),26194,16820,74.18,60.9
73,India (Overall),22929,14957,65.34,60.5
57,North America (Overall),5059,3482,14.73,59.2
0,Total Overall,33234,24752,100.00,57.3
11,Australasia (Overall),518,489,1.74,51.4


In [26]:
vote_total_table.to_csv("2016_election_table.csv", index=False)